## Límites de permisos de IAM delegando la creación de roles

Este laboratorio práctico le guiará a través de los pasos para configurar un ejemplo de límite de permisos de AWS Identity and Access Management (IAM). AWS admite límites de permisos para entidades de IAM (usuarios o roles). Un límite de permisos es una función avanzada en la que se utiliza una política administrada para establecer los permisos máximos que una política basada en identidades puede conceder a una entidad de IAM. Cuando se establece un límite de permisos para una entidad, la entidad sólo puede realizar las acciones permitidas por la política. En este laboratorio usted creará una serie de políticas adjuntas a un rol que puede ser asumido por un individuo tal como un desarrollador, el desarrollador puede entonces usar este rol para crear roles de usuario adicionales que están restringidos a servicios y regiones específicas. Esto le permite delegar el acceso para crear roles y políticas de IAM, sin que excedan los permisos en el límite de permisos. También utilizaremos un estándar de nomenclatura con un prefijo, lo que facilitará el control y la organización de las políticas y roles que creen los desarrolladores.

In [3]:
import boto3
import json
from botocore.exceptions import ClientError

iam = boto3.client('iam')

Esta política se utilizará para el límite de permisos cuando el rol de desarrollador cree su propio rol de usuario con sus permisos delegados. En este laboratorio usando AWS IAM solo vamos a permitir las regiones us-east-1 (Norte de Virginia) y us-west-1 (Norte de California). Las únicas acciones de servicio que vamos a permitir en estas regiones son AWS EC2 y AWS Lambda, ten en cuenta que estos servicios requieren acciones de soporte adicionales si fueras a reutilizar esta política después de este laboratorio, dependiendo de tus necesidades.

In [4]:
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "EC2RestrictRegion",
            "Effect": "Allow",
            "Action": "ec2:*",
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "aws:RequestedRegion": [
                        "us-east-1",
                        "us-west-1"
                    ]
                }
            }
        },
        {
            "Sid": "LambdaRestrictRegion",
            "Effect": "Allow",
            "Action": "lambda:*",
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "aws:RequestedRegion": [
                        "us-east-1",
                        "us-west-1"
                    ]
                }
            }
        }
    ]
}


In [5]:
policy_name = 'restrict-region-boundary'
response = iam.create_policy(
        PolicyName=policy_name,
        PolicyDocument=json.dumps(policy)
)
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print("Policy created successfully")

Policy created successfully


### Crear política restringida IAM de desarrollador

Esta política se adjuntará al rol de desarrollador, y permitirá al desarrollador crear políticas y roles con un prefijo de nombre app1, y sólo si se adjunta el límite de permiso restrict-region-boundary. Los prefijos de nombres son útiles cuando tienes diferentes equipos o, en este caso, diferentes aplicaciones ejecutándose en la misma cuenta de AWS. Pueden ser utilizados para mantener sus recursos de aspecto ordenado, y también en la política de IAM como el recurso como lo estamos haciendo aquí.

In [7]:
policy2 = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "CreatePolicy",
            "Effect": "Allow",
            "Action": [
                "iam:CreatePolicy",
                "iam:CreatePolicyVersion",
                "iam:DeletePolicyVersion"
            ],
            "Resource": "arn:aws:iam::089715336747:policy/app1*"
        },
        {
            "Sid": "CreateRole",
            "Effect": "Allow",
            "Action": [
                "iam:CreateRole"
            ],
            "Resource": "arn:aws:iam::089715336747:role/app1*",
            "Condition": {
                "StringEquals": {
                    "iam:PermissionsBoundary": "arn:aws:iam::089715336747:policy/restrict-region-boundary"
                }
            }
        },
        {
            "Sid": "AttachDetachRolePolicy",
            "Effect": "Allow",
            "Action": [
                "iam:DetachRolePolicy",
                "iam:AttachRolePolicy"
            ],
            "Resource": "arn:aws:iam::089715336747:role/app1*",
            "Condition": {
                "ArnEquals": {
                    "iam:PolicyARN": [
                        "arn:aws:iam::089715336747:policy/*",
                        "arn:aws:iam::aws:policy/*"
                    ]
                }
            }
        }      
    ]
}


In [8]:
policy_name2 = 'createrole-restrict-region-boundary'
response = iam.create_policy(
        PolicyName=policy_name2,
        PolicyDocument=json.dumps(policy2)
)
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print("Policy created successfully")

Policy created successfully


### Crear una política de acceso a la consola IAM para desarrolladores

Esta política permite acciones de servicio IAM de tipo lista y lectura para que puedas ver lo que has creado usando la consola.

In [9]:
policy3 = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "Get",
            "Effect": "Allow",
            "Action": [
                "iam:ListPolicies",
                "iam:GetRole",
                "iam:GetPolicyVersion",
                "iam:ListRoleTags",
                "iam:GetPolicy",
                "iam:ListPolicyVersions",
                "iam:ListAttachedRolePolicies",
                "iam:ListRoles",
                "iam:ListRolePolicies",
                "iam:GetRolePolicy"
            ],
            "Resource": "*"
        }
    ]
}

In [11]:
role_name3 = 'iam-restricted-list-read'
response = iam.create_policy(
    PolicyName=role_name3,
    PolicyDocument=json.dumps(policy3)
)
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print("Policy created successfully")

Policy created successfully


In [ ]:
policy_name = 'dynamodb-policy-test-sample'
try:
    response = iam.create_policy(
        PolicyName=policy_name,
        PolicyDocument=json.dumps(policy)
    )
    print(f"Policy {policy_name} created successfully. Arn: {response['Policy']['Arn']}")

except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        #Update policy
        account_id = boto3.client('sts').get_caller_identity().get('Account')
        response = iam.get_policy(PolicyArn=f"arn:aws:iam::{account_id}:policy/{policy_name}")
        policy_arn = response['Policy']['Arn']
        # Delete all policy versions except the default version
        response = iam.list_policy_versions(PolicyArn=f"arn:aws:iam::{account_id}:policy/{policy_name}")
        for version in response['Versions']:
            if not version['IsDefaultVersion']:
                response = iam.delete_policy_version(
                    PolicyArn=policy_arn,
                    VersionId=version['VersionId']
                )   
        response = iam.create_policy_version(
            PolicyArn=policy_arn,
            PolicyDocument=json.dumps(policy),
            SetAsDefault=True
        )
        print(f"Policy {policy_name} updated successfully. Arn: {policy_arn}")
    else:
        print("Unexpected error: %s" % e)